In [ ]:
import psycopg2
from fractions import Fraction
class PostgresConnection:
    _connection = None
    _cursor = None

    @staticmethod
    def connect():
        if PostgresConnection._connection is None:
            try:
                PostgresConnection._connection = psycopg2.connect(
                    host="43.207.238.120",
                    dbname="stagegraph_simulations",
                    user="postgres",
                    password="P)stGres@321!"
                )
                PostgresConnection._cursor = PostgresConnection._connection.cursor()
                print("Connection established.")
            except Exception as e:
                print(f"Error connecting to PostgreSQL database: {e}")

    @staticmethod
    def execute_query(query, params=None):
        if PostgresConnection._connection is None:
            PostgresConnection.connect()
        try:
            PostgresConnection._cursor.execute(query, params)
            PostgresConnection._connection.commit()
        except Exception as e:
            print(f"Error executing query: {query}\nError: {e}")

    @staticmethod
    def fetch_data(query, params=None):
        if PostgresConnection._connection is None:
            PostgresConnection.connect()
        try:
            PostgresConnection._cursor.execute(query, params)
            return PostgresConnection._cursor.fetchall()
        except Exception as e:
            print(f"Error fetching data: {e}")
            return None

    @staticmethod
    def close_connection():
        if PostgresConnection._cursor:
            PostgresConnection._cursor.close()
        if PostgresConnection._connection:
            PostgresConnection._connection.close()
        PostgresConnection._connection = None
        PostgresConnection._cursor = None
        print("Connection closed.")


This is First Sheet

Sheet no 2

In [ ]:
# # '2024-08-26 17:30:00' AND created_at <= '2024-08-26 18:00:00'
# date = '2024-08-31 16:41'
# endTime = '2024-08-31 17:11' #it can be None

date = '2024-08-31 08:00'
endTime = '2024-08-31 09:10'
from datetime import datetime

# Get the current date and time
current_date_time = datetime.now()
# '2024-08-25 14:34:00' AND start_time <= '2024-08-25 15:44:00'
# Format the date and time in the desired format
formatted_date_time = current_date_time.strftime("%Y:%m:%d:%H:%M")
filename = f"1.001.xlsx"
# where start_time > '2024-08-24 10:00:00'

# filename = f"output.xlsx"
import pandas as pd

from decimal import Decimal

def calculate_end_amount(stages, initial_amount):
    current_amount = Decimal(initial_amount)
    end_amount = current_amount

    for stage in stages:
        rate = Decimal(stage["rate"])
        end_amount *= rate
    end_amount = end_amount.quantize(Decimal('0.0001'))

    return end_amount

def calculate_profit(stages, initial_amount):
    end_amount = calculate_end_amount(stages, initial_amount)
    return end_amount - Decimal(initial_amount)

def extract_client_order_ids(stages):
    return ", ".join([stage["client_order_id"] for stage in stages])

def getOrderType(order):
    return order.get("order_type")

def getSymbol(order_response):
    return order_response.get("symbol")

def getSide(order):
    return order.get("side")

def getQuantity(order):
  try:
    return order.get("quantity" , None)
  except:
    return None

def getExecutedQuantity(order_response):
    try:
      return order_response.get("executedQty", None)
    except:
      return None

def getStageNumber(row):
    for stage in row["stage"]:
        if stage.get("client_order_id") + "_SP" == row["client_order_id"]:
            return stage.get("stage_no")
    return None

def getExchange(order):
    exchange_full = order.get("exchanges", [None])[0]
    return exchange_full.replace('sim_', '') if exchange_full else None


def getGrossConsideration(order_response):
    if order_response is None:
      return ""
    fills = order_response.get("fills", [])

    if not fills:
        return 0.0  # Return 0.0 instead of a string "0" for consistency in numeric operations

    gross_consideration = sum(
        float(fill.get("price", 0)) * float(fill.get("qty", 0)) for fill in fills
    )

    return gross_consideration

def getRealizedprofit(df, row, strategyID, start_coin_quantity, net_consideration):
    # Filter the DataFrame where unique_strategy_id matches strategyID
    strategy_filtered_df = df[df["unique_strategy_id"] == strategyID]
    # filtered_df = database_DF[database_DF["unique_strategy_id"] == unique_strategy_id]
    strategy_filtered_df  =  strategy_filtered_df .reset_index(drop=True)
    # Check if strategy_filtered_df is not empty
    if not strategy_filtered_df.empty:
        # Get the last net_consideration value
        last_net_consideration = strategy_filtered_df.iloc[-1]
        rowCreatedTime = row['created_at']
        matching_indices = strategy_filtered_df.index[strategy_filtered_df['created_at'] == rowCreatedTime].tolist()

        index = matching_indices[0]
        # last_index = len(filtered_df) - 1

            # If the current row is the first row and there's more than one row
        if index == 0 and len( strategy_filtered_df) > 1:
            # Get the first net_consideration value
            first_net_consideration = strategy_filtered_df.iloc[0]["Net Consideration"]
            last_net_consideration_value = last_net_consideration["Net Consideration"]
            temp1 = row["unique_strategy_id"]
            temp2 = last_net_consideration["unique_strategy_id"]
            orderRes = last_net_consideration["order_response"]
            orderRes2 = row["order_response"]
            # print(f"{last_net_consideration_value} ,, {first_net_consideration} {temp2}")
            # Convert values to float if necessary
            print(f"first_net_consideration = {type(first_net_consideration)} , start_coin_quantity {type(start_coin_quantity)}")
            first_net_consideration = Fraction(str(first_net_consideration))
            last_net_consideration_value = Fraction(str(last_net_consideration_value))
            start_coin_quantity = Fraction(str(start_coin_quantity))
            print(f"profit = {float(last_net_consideration_value - first_net_consideration)}")
            # Perform the calculation
            return float(last_net_consideration_value - first_net_consideration)
        else:
            return ""  # No matching start_time found
    else:
        return ""  # No matching strategyID found



def getNetConsideration(order_response):
    if order_response is None:
      return ""
    # Step 1: Extract 'fills' from the order_response
    fills = order_response.get("fills", [])
    # print(f"Order Response: {order_response} {type(order_response)}")

    # print("Fills extracted:", fills)

    # Step 2: Check if 'fills' is empty
    if not fills:
        # If no fills are found, log and return 0.0
        # print("No fills found. Returning 0.0")
        return 0.0

    # Step 3: Initialize net_consideration variable to sum up all the calculated values
    net_consideration = 0.0

    # Step 4: Iterate over each fill and calculate net consideration
    for index, fill in enumerate(fills):
        try:
            # price = float(fill.get("price", 0.0))
            # qty = float(fill.get("qty", 0.0))
            # commission = float(fill.get("commission", 0.0))
            price = Fraction(str(fill.get("price", 0.0)))
            qty = Fraction(str(fill.get("qty", 0.0)))
            commission = Fraction(str(fill.get("commission", 0.0)))
            print(f"prince = {price} , qty {qty} , commision = {commission}")
        except ValueError as e:
            print(f"Error converting fill data to float at index {index}: {fill}")
            continue

        # print(f"Processing fill {index}: price={price}, qty={qty}, commission={commission}")

        gross_consideration = price * qty
        # print(f"Gross consideration for fill {index}: {gross_consideration}")

        net_fill_consideration = gross_consideration - commission
        # print(f"Net consideration for fill {index}: {net_fill_consideration}")

        net_consideration += net_fill_consideration
        # print(f"Updated net consideration after fill {index}: {net_consideration}")

    print(f"Final net consideration: {net_consideration}")
    return net_consideration


def setExchangeQuantites(row, symbol, exchange, database_DF):
    unique_strategy_id = row['unique_strategy_id']
    filtered_df = database_DF[database_DF["unique_strategy_id"] == unique_strategy_id]
    filtered_df = filtered_df.reset_index(drop=True)
    order = row.get("order", {})
    order_Response = row.get("order_response", {})
    Rowsymbol = order.get("symbol", "")
    Rowexchange = order.get("exchanges", [None])[0].split('_')[-1] if order.get("exchanges") else None
    RowSide = order.get("side")
    client_orderId = order.get("client_order_id", None)
    quantity = getExecutedQuantity(order_Response)
    # print(RowSide)
    rowCreatedTime = row['created_at']
    if Rowsymbol.startswith(symbol) and exchange == Rowexchange:
        if quantity is not None:
            if RowSide.lower() == "sell":
                # print("side = sell")
                matching_indices = filtered_df.index[filtered_df['created_at'] == rowCreatedTime].tolist()
                index = matching_indices[0]
                last_index = len(filtered_df) - 1
                # print(f"{index} - last index = {last_index}")
                # if matching_indices:
                if index == last_index:
                    return ""
                return f"-{quantity}"
            else:
              matching_indices = filtered_df.index[filtered_df['created_at'] == rowCreatedTime].tolist()
              return f"{quantity}"
              # Check if we found a matching index
              if matching_indices:
                  index = matching_indices[0]
                  last_index = len(filtered_df) - 1

                  # If the current row is the first row and there's more than one row
                  if index == 0 and len(filtered_df) > 1:
                    return ""
              return f"{quantity}"
        else:
            return ""
    else:
        # Get the index of the row where 'created_at' matches 'rowCreatedTime'
        matching_indices = filtered_df.index[filtered_df['created_at'] == rowCreatedTime].tolist()

        # Check if we found a matching index
        if matching_indices:
            index = matching_indices[0]
            last_index = len(filtered_df) - 1

            # If the current row is the first row and there's more than one row
            if index == 0 and len(filtered_df) > 1:
                # Get the next row
                nextRow = filtered_df.iloc[1]
                nextRoworder = nextRow.get("order", {})
                nextRowSymbol = nextRoworder.get("symbol", "")
                nextRowexchange = (
                    nextRoworder.get("exchanges", [None])[0].split('_')[-1]
                    if nextRoworder.get("exchanges")
                    else None
                )

                # Compare exchanges and symbols
                # if nextRowexchange != Rowexchange:
                nextRowOrderResponse = nextRow.get("order_response", {})
                # nextRowSymbol = prevRoworder.get("symbol", "")
                nextRowQuantity = getExecutedQuantity(nextRowOrderResponse)
                if exchange == Rowexchange and  Rowsymbol.startswith(symbol):
                    return f"{quantity}"
                # if exchange == nextRowexchange and nextRowSymbol.startswith(symbol):
                #     return f"{nextRowQuantity}"

            # If the current row is the last row
            elif index == last_index:
                return ""
                # if Rowsymbol.startswith(symbol) and exchange == Rowexchange:
                #     if quantity is not None:
                #         return f"-{quantity}"

            # For other rows, check the previous row
            else:
                prevRow = filtered_df.iloc[index - 1]
                prevRoworder = prevRow.get("order", {})
                prevOrderResponse = prevRow.get("order_response", {})
                prevRowSymbol = prevRoworder.get("symbol", "")
                prevRowQuantity = getExecutedQuantity(prevOrderResponse)
                prevRowexchange = (
                    prevRoworder.get("exchanges", [None])[0].split('_')[-1]
                    if prevRoworder.get("exchanges")
                    else None
                )

                # Compare exchanges and symbols
                if exchange == prevRowexchange and prevRowSymbol.startswith(symbol):
                    if quantity is not None:
                        return f"-{prevRowQuantity}"

        # If no conditions match, return an empty string
        return ""



def setExchangeQuantites2(row, symbol, exchange, database_DF):
    unique_strategy_id = row['unique_strategy_id']
    filtered_df = database_DF[database_DF["unique_strategy_id"] == unique_strategy_id]
    filtered_df = filtered_df.reset_index(drop=True)
    order = row.get("order", {})
    order_Response = row.get("order_response", {})
    Rowsymbol = order.get("symbol", "")
    Rowexchange = order.get("exchanges", [None])[0].split('_')[-1] if order.get("exchanges") else None
    RowSide = order.get("side")
    client_orderId = order.get("client_order_id", None)
    quantity = getExecutedQuantity(order_Response)
    rowCreatedTime = row['created_at']
    matching_indices = filtered_df.index[filtered_df['created_at'] == rowCreatedTime].tolist()[0]
    last_index = len(filtered_df) - 1

    if matching_indices == 0:

      nextRow = filtered_df.iloc[matching_indices+1]
      nextRoworder = nextRow.get("order", {})
      nextRowSymbol = nextRoworder.get("symbol", "")
      nextRowexchange = (
                      nextRoworder.get("exchanges", [None])[0].split('_')[-1]
                      if nextRoworder.get("exchanges")
                      else None
                  )

                  # Compare exchanges and symbols
                  # if nextRowexchange != Rowexchange:
      nextRowOrderResponse = nextRow.get("order_response", {})
                  # nextRowSymbol = prevRoworder.get("symbol", "")
      nextRowQuantity = getExecutedQuantity(nextRowOrderResponse)
      if nextRowexchange != Rowexchange:
        if exchange == Rowexchange and  Rowsymbol.startswith(symbol):
          return f"-{quantity}"
        elif exchange == nextRowexchange and  Rowsymbol.startswith(symbol):
            return f"{quantity}"

    else:
      if last_index == matching_indices:
                return ""

      nextRow = filtered_df.iloc[matching_indices+1]
      nextRoworder = nextRow.get("order", {})
      nextRowSymbol = nextRoworder.get("symbol", "")
      nextRowexchange = (
                      nextRoworder.get("exchanges", [None])[0].split('_')[-1]
                      if nextRoworder.get("exchanges")
                      else None
                  )

                  # Compare exchanges and symbols
                  # if nextRowexchange != Rowexchange:
      nextRowOrderResponse = nextRow.get("order_response", {})
                  # nextRowSymbol = prevRoworder.get("symbol", "")
      nextRowQuantity = getExecutedQuantity(nextRowOrderResponse)
      if nextRowexchange != Rowexchange:
        if exchange == Rowexchange and  Rowsymbol.endswith(symbol):
          return f"-{nextRowQuantity}"
        elif exchange == nextRowexchange and  Rowsymbol.endswith(symbol):
            return f"{nextRowQuantity}"

def setDust(row, symbol, database_DF):
    unique_strategy_id = row['unique_strategy_id']
    filtered_df = database_DF[database_DF["unique_strategy_id"] == unique_strategy_id]
    filtered_df = filtered_df.reset_index(drop=True)

    order = row.get("order", {})
    order_response = row.get("order_response", {})
    row_symbol = order.get("symbol", "")
    row_side = order.get("side")
    client_order_id = order.get("client_order_id", None)
    quantity = Fraction(str(getExecutedQuantity(order_response)))
    row_created_time = row['created_at']

    matching_indices = filtered_df.index[filtered_df['created_at'] == row_created_time].tolist()
    if not matching_indices:
        return None

    matching_index = matching_indices[0]
    last_index = len(filtered_df) - 1

    current_df = filtered_df.loc[matching_index]
    current_net = Fraction(str(current_df["Net Consideration"]))
    qty = Fraction(str(row.get('Net Executed Qty', '0')))

    if matching_index == 0:
        if row_symbol.endswith(symbol):
            start_qty = Fraction(str(current_df["start_coin_quantity"]))
            net = Fraction(str(current_df["Net Consideration"]))
            return ""

    else:
        # next_df = filtered_df.loc[matching_index + 1]
        last_df = filtered_df.loc[matching_index - 1]

        last_qty = Fraction(str(last_df["Net Executed Qty"]))
        last_net = Fraction(str(last_df["Net Consideration"]))
        # next_net = Fraction(str(next_df["Net Consideration"]))
        # next_qty = Fraction(str(next_df["Net Executed Qty"]))

        last_order = last_df["order"]
        last_side = last_order.get("side", "")

        # next_order = next_df["order"]
        # next_side = next_order.get("side", "")

        # print(f"\nNext Net: {next_net}, Current Qty: {qty}")
        # print(f"Row Side: {row_side}, Next Side: {next_side}")

        # dust = qty - next_net

        if row_side == "sell" and last_side == "buy":
            if row_symbol.startswith(symbol):
                dust = last_qty - qty
                return float(dust)
        elif row_side == "sell" and last_side == "sell":
            if row_symbol.startswith(symbol):
                dust = last_net - qty
                return float(dust)
        elif row_side == "buy" and last_side == "buy":
            if row_symbol.startswith(symbol):
              dust = last_qty - current_net
              return float(dust)

    return None


def setCycleProfitProfit(row, reconsile):

    unique_strategy_id = row.get("Cycle ID")
    filtered_df = reconsile[reconsile["Cycle ID"] == unique_strategy_id]
    filtered_df = filtered_df.reset_index(drop=True)

    if not filtered_df.empty:
        temp = filtered_df.loc[0]
        realized_profit = filtered_df.loc[0, "Realized Profit (USD)"]
        return realized_profit
    else:
        # Handle the case where no rows match the criteria
        return None

PostgresConnection.connect()

# Fetch data for CycleProfitSummery
query = f"select * from strategy_log where start_time > '{date}' order by start_time asc;"
if endTime is not None:
  query = f"select * from strategy_log where start_time > '{date}' AND start_time <= '{endTime}' order by start_time asc;"

data = PostgresConnection.fetch_data(query)
Strategy_LogColumns = [
    "number_of_stages", "longest_path_length", "start_time", "end_time",
    "start_token", "end_token", "stage", "test_rate", "validate",
    "unique_strategy_id", "trader_id", "start_coin_quantity"
]
Strategy_LogDF = pd.DataFrame(data, columns=Strategy_LogColumns)

# Initialize CycleProfitSummery DataFrame
CycleProfitSummery = pd.DataFrame({
    "Cycle ID": Strategy_LogDF["unique_strategy_id"],
    "Started At": Strategy_LogDF["start_time"],
    "Finished At": Strategy_LogDF["end_time"],
    "Cycle Time":  (Strategy_LogDF["end_time"] - Strategy_LogDF["start_time"]).dt.total_seconds() / 86400,
    "Stages": Strategy_LogDF["stage"].apply(len),
    "Base Currency": Strategy_LogDF["start_token"],
    "Start Amount": Strategy_LogDF["start_coin_quantity"],
    "End Amount": Strategy_LogDF.apply(lambda row: calculate_end_amount(row["stage"], row["start_coin_quantity"]), axis=1),
    "Profit": "Hello",
    "Parent Order IDs": Strategy_LogDF["stage"].apply(extract_client_order_ids)
})

# CycleProfitSummery.head(500)

# Fetch data for CycleParentOrder
query = f"""
SELECT *
FROM strategy_log sl
INNER JOIN strategy_order_log sol
ON sl.unique_strategy_id = sol.unique_strategy_id
where created_at >  '{date}'
order by created_at asc;

"""
if endTime is not None:
  query = f"""
    SELECT *
    FROM strategy_log sl
    INNER JOIN strategy_order_log sol
    ON sl.unique_strategy_id = sol.unique_strategy_id
    where created_at >  '{date}' and created_at <= '{endTime}'
    order by created_at asc;
    """
  # query = f"""SELECT *
  #   FROM strategy_log sl
  #   INNER JOIN strategy_order_log sol
  #   ON sl.unique_strategy_id = sol.unique_strategy_id
  #   WHERE sl.unique_strategy_id = '1725092100206844528_3_order'
  #   ORDER BY created_at ASC;
  #   """
print(query)
# query = f"""
# SELECT *
# FROM strategy_log sl
# INNER JOIN strategy_order_log sol
# ON sl.unique_strategy_id = sol.unique_strategy_id
# where sl.unique_strategy_id =  '1724509682213416417_2_order'

# """

# Cycle ID
data = PostgresConnection.fetch_data(query)
dataBasecolumns = [
    "number_of_stages", "longest_path_length", "start_time", "end_time",
    "start_token", "end_token", "stage", "test_rate", "validate", "unique_strategy_id",
    "trader_id", "start_coin_quantity", "client_order_id", "order", "order_response",
    "order_reject", "created_at", "old_client_order_id", "unique_strategy_id-2",
    "error_type", "error_txt"
]
database_DF = pd.DataFrame(data, columns=dataBasecolumns)
# Sorting the DataFrame by "Cycle ID"
CycleParentOrder = database_DF.sort_values(by="unique_strategy_id-2")

# Initialize CycleParentOrder DataFrame
CycleParentOrder = pd.DataFrame({
    "Cycle ID": database_DF["unique_strategy_id"],
    "Started At": database_DF["start_time"],
    "Finished At": database_DF["end_time"],
    "Order Type": database_DF["order"].apply(getOrderType),
    "Currency Pair": database_DF["order_response"].apply(getSymbol),
    "Exchange": database_DF["order"].apply(getExchange),
    "Side": database_DF["order"].apply(getSide),
    "PCT Limit Away": database_DF["test_rate"],
    "Quantity": database_DF["order"].apply(getQuantity),
    "Executed Qty": database_DF["order_response"].apply(getExecutedQuantity),
    "Net Executed Quantity": database_DF["order_response"].apply(getExecutedQuantity), #// minus commision
    "Gross Consideration": database_DF["order_response"].apply(getGrossConsideration),
    "Net Consideration": database_DF["order_response"].apply(getNetConsideration),
    "Owner Username": database_DF["trader_id"],
    "Strategy Slug": database_DF["unique_strategy_id"],
    "Client Order Id": database_DF["client_order_id"],
    "External Order Customer": database_DF["old_client_order_id"],
    "Stage No": database_DF.apply(getStageNumber, axis=1)
})
# CycleParentOrder.head(500)

# # Create a DataFrame for reconciliation
filtered_symbols = database_DF["order"].apply(lambda x: x["symbol"]).unique()
symbols = [symbol.split('-')[0] for symbol in filtered_symbols if "USDT" in symbol]

unique_exchanges = database_DF["order"].apply(lambda x: x["exchanges"]).explode().unique()
exchanges = [exchange.replace('sim_', '') for exchange in unique_exchanges]

reconsile_Coloums = [
    'Cycle ID', 'Created At', 'Exchange', 'Currency Pair', 'Side',
    'Net Executed Qty', 'Net Consideration', 'Realized Profit (USD)'
]

for exchange in exchanges:
    for symbol in symbols:
        reconsile_Coloums.append(exchange + "_" + symbol)

for symbol in symbols:
    reconsile_Coloums.append("Dust_" + symbol)

reconsile = pd.DataFrame(columns=reconsile_Coloums)

reconsile["Cycle ID"] = database_DF["unique_strategy_id"]
reconsile["Created At"] = database_DF["start_time"]
reconsile["Exchange"] = database_DF["order"].apply(getExchange)
reconsile["Currency Pair"] = database_DF["order_response"].apply(getSymbol)
reconsile["Side"] = database_DF["order"].apply(getSide)
reconsile["Net Executed Qty"] = database_DF["order_response"].apply(getExecutedQuantity)
reconsile["Net Consideration"] = database_DF["order_response"].apply(getNetConsideration)
database_DF["Net Consideration"] =reconsile["Net Consideration"]
database_DF["Net Executed Qty"] =reconsile["Net Executed Qty"]

reconsile["Realized Profit (USD)"] = database_DF.apply(
    lambda row: getRealizedprofit(database_DF,row, row["unique_strategy_id"], row["start_coin_quantity"] ,row["Net Consideration"]  ),
    axis=1
)
# --Calculating Exchanges result
for exchange in exchanges:
    for symbol in symbols:
        column_name = exchange + "_" + symbol
        reconsile[column_name] = database_DF.apply(
            lambda row: setExchangeQuantites2(row, symbol, exchange, database_DF),
            axis=1
        )
#Calculating Exchanges result result
symbols.append("USDT")







for symbol in symbols:
    column_name = "Dust_"+symbol
    reconsile[column_name] = database_DF.apply(
    lambda row: setDust(row, symbol , database_DF),
    axis=1
        )
sumjson = {}
for col in reconsile_Coloums:
  skipedreconsile_Coloums = [
      'Cycle ID', 'Created At', 'Exchange', 'Currency Pair', 'Side',
      'Net Executed Qty', 'Net Consideration'
  ]
  if col in skipedreconsile_Coloums:
      sumjson[col] = ""
  else:
    reconsile[col] = pd.to_numeric(reconsile[col], errors='coerce')

    # Calculate the sum, ignoring NaN values
    sum_val = reconsile[col].sum()

    # Store the sum in the dictionary
    sumjson[col] = sum_val

# Create a Pandas Excel writer using openpyxl as the engine
CycleProfitSummery["Profit"] = CycleProfitSummery.apply(
    lambda row: setCycleProfitProfit(row,  reconsile),
    axis=1
        )
reconsile.head()
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    # Write each DataFrame to a different sheet
    CycleProfitSummery.to_excel(writer, sheet_name='CycleProfitSummary', index=False)
    CycleParentOrder.to_excel(writer, sheet_name='CycleParentOrder', index=False)


    grouped_df = reconsile.groupby('Cycle ID', as_index=False).apply(lambda x: x).reset_index(drop=True)

    # Create a list to store DataFrame chunks
    df_list = []

    # Iterate over each group and add a blank row after each group
    for cycle_id, group in grouped_df.groupby('Cycle ID'):
        df_list.append(group)
        # Append a blank row (NaNs)
        df_list.append(pd.DataFrame([[''] * len(reconsile.columns)], columns=reconsile.columns))

    # Concatenate all chunks into a final DataFrame
    final_df = pd.concat(df_list, ignore_index=True)

    # # Save to Excel
    # final_df.to_excel('cycle_summary.xlsx', index=False)

    # Convert sumjson to a Series
    # sum_series = pd.Series(sumjson)
    sum_df = pd.DataFrame([sumjson])
    # Add the Series as a new row to the DataFrame
    final_df = pd.concat([final_df, sum_df], )
    final_df.to_excel(writer, sheet_name='Reconcile', index=False)


    # workbook  = writer.book
    # worksheet = writer.sheets['Reconsile']

    # # Optional: Format the last row to distinguish it (e.g., bold font)
    # last_row = len(final_df)  # Index of the last row
    # worksheet.write(last_row, 0, 'Total', workbook.add_format({'bold': True}))
    # worksheet.write(last_row, 1, "52", workbook.add_format({'bold': True}))

# The Excel file is saved automatically when the 'with' block is exited
print(f"Excel file '{filename}' created successfully.")



    SELECT *
    FROM strategy_log sl
    INNER JOIN strategy_order_log sol
    ON sl.unique_strategy_id = sol.unique_strategy_id
    where created_at >  '2024-08-31 08:00' and created_at <= '2024-08-31 09:10'
    order by created_at asc;
    
prince = 1739/5000 , qty 71880391/25000 , commision = 0
Final net consideration: 999.999999592
prince = 349/1000 , qty 71880391/25000 , commision = 0
Final net consideration: 1003.45025836
prince = 126141/50 , qty 19819/50000 , commision = 0
Final net consideration: 999.9953916
prince = 1067/25000 , qty 19819/50000 , commision = 0
Final net consideration: 0.0169174984
prince = 59128 , qty 423/25000 , commision = 0
Final net consideration: 1000.44576
prince = 3483/10000 , qty 143554407/50000 , commision = 0
Final net consideration: 999.999999162
prince = 1381/10000000 , qty 143554407/50000 , commision = 0
Final net consideration: 0.396497272134
prince = 2523 , qty 793/2000 , commision = 0
Final net consideration: 1000.3695
prince = 3483/10000 , q